In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/augmented_corrected2.zip', 'r') as zip_ref:
    zip_ref.extractall()

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense , Dropout , Flatten
from keras.layers import Conv2D , MaxPooling2D
import os
import cv2
import tensorflow as tensorflow
import matplotlib.pyplot as plt
import numpy as np

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
import os
import tqdm
import glob
import tensorflow

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from skimage.io import imread, imshow
from skimage.transform import resize

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.densenet import DenseNet169
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from tensorflow.keras.layers import Conv2D , Flatten , MaxPooling2D , BatchNormalization , Dense , Dropout , Activation , MaxPool2D

import os
import random
import shutil






In [3]:
folder_path = '/content/content/augmented_corrected2/.ipynb_checkpoints'

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print("Folder deleted successfully.")
else:
    print("Folder not found.")

Folder deleted successfully.


In [6]:

! pip install split-folders

import splitfolders
import os
path = "/content/content/augmented_corrected2"
print(os.listdir(path))

import shutil
import os



splitfolders.ratio(path,seed=1337, output="data-Splitted101", ratio=(0.70, 0.20, 0.10))

['smile', 'neutral', 'eye_contact', 'surprise', 'happy', 'sad', 'angry']


Copying files: 42000 files [00:06, 6682.55 files/s]


In [8]:
train_datagen = ImageDataGenerator(rescale = 1./255,

        zoom_range=0.3,
        horizontal_flip=True,
        fill_mode='nearest'  )

valid_datagen = ImageDataGenerator(rescale = 1./255,
                                   )

test_datagen  = ImageDataGenerator(rescale = 1./255)

import shutil
import os



train_dataset  = train_datagen.flow_from_directory(directory = r'/content/data-Splitted101/train',
                                                   target_size = (224,224),
                                                   class_mode = 'categorical',

                                                   batch_size = 128)

valid_dataset = valid_datagen.flow_from_directory(directory = r'/content/data-Splitted101/val',
                                                  target_size = (224,224),
                                                  class_mode = 'categorical',

                                                  batch_size = 128)



Found 29400 images belonging to 7 classes.
Found 8400 images belonging to 7 classes.


In [9]:
from tensorflow.keras.models import model_from_json


# from keras import model_from_json

# opening and store file in a variable

json_file = open('/content/drive/MyDrive/modeldensenet169_100.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

# use Keras model_from_json to make a loaded model
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("/content/drive/MyDrive/best_weights_for densenetmodel7classess100.hdf5")
print("Loaded Model from disk")

# compile and evaluate loaded model

# Model Initialization densenet169




Loaded Model from disk


In [10]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet169 (Functional)    (None, 7, 7, 1664)        12642880  
                                                                 
 dropout (Dropout)           (None, 7, 7, 1664)        0         
                                                                 
 flatten (Flatten)           (None, 81536)             0         
                                                                 
 batch_normalization (Batch  (None, 81536)             326144    
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 2048)              166987776 
                                                                 
 batch_normalization_1 (Bat  (None, 2048)              8192      
 chNormalization)                                       

In [23]:

model1 = Sequential()
for layer in loaded_model.layers[:-1]: # go through until last layer
    model1.add(layer)


In [24]:
model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet169 (Functional)    (None, 7, 7, 1664)        12642880  
                                                                 
 dropout (Dropout)           (None, 7, 7, 1664)        0         
                                                                 
 flatten (Flatten)           (None, 81536)             0         
                                                                 
 batch_normalization (Batch  (None, 81536)             326144    
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 2048)              166987776 
                                                                 
 batch_normalization_1 (Bat  (None, 2048)              8192      
 chNormalization)                                     

In [25]:
model1.add(Dense(1028,kernel_initializer='he_uniform' ,   name = 'added_dense1'))
model1.add(Dense(1028,kernel_initializer='he_uniform' ,   name = 'added_dense2'))

model1.add(Dense(7,name = "dense_last_23"  , activation='softmax'))

In [26]:
model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet169 (Functional)    (None, 7, 7, 1664)        12642880  
                                                                 
 dropout (Dropout)           (None, 7, 7, 1664)        0         
                                                                 
 flatten (Flatten)           (None, 81536)             0         
                                                                 
 batch_normalization (Batch  (None, 81536)             326144    
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 2048)              166987776 
                                                                 
 batch_normalization_1 (Bat  (None, 2048)              8192      
 chNormalization)                                     

In [27]:
# Model Compile

OPT    = tensorflow.keras.optimizers.Adam(lr=0.0001)
#tensorflow.keras.optimizers.Adam(learning_rate=0.0001)

model1.compile(loss='categorical_crossentropy',
              metrics=[tensorflow.keras.metrics.AUC(name = 'auc')],
              optimizer=OPT)

# Defining Callbacks
import math
filepath = './best_weights_densenet169_401classes.hdf5'

earlystopping = EarlyStopping(monitor = 'val_loss',
                             patience=5
                              )

checkpoint    = ModelCheckpoint(filepath,
                                monitor = 'val_auc',
                                mode='max',
                                save_best_only=True,
                                verbose = 1)


callback_list = [earlystopping, checkpoint]

model_history_densenet169=model1.fit(train_dataset,
                        validation_data=valid_dataset,
                        epochs = 15,
                        steps_per_epoch= train_dataset.samples//train_dataset.batch_size
                        , validation_steps=valid_dataset.samples//valid_dataset.batch_size,
                        callbacks = callback_list,
                        verbose = 1)

Epoch 1/15
229/229 [==============================] - ETA: 0s - loss: 1.8054 - auc: 0.8036
Epoch 1: val_auc improved from -inf to 0.88711, saving model to ./best_weights_densenet169_401classes.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


229/229 [==============================] - 511s 2s/step - loss: 1.8054 - auc: 0.8036 - val_loss: 1.1623 - val_auc: 0.8871
Epoch 2/15
229/229 [==============================] - ETA: 0s - loss: 1.2753 - auc: 0.8684
Epoch 2: val_auc improved from 0.88711 to 0.90264, saving model to ./best_weights_densenet169_401classes.hdf5
229/229 [==============================] - 461s 2s/step - loss: 1.2753 - auc: 0.8684 - val_loss: 1.0694 - val_auc: 0.9026
Epoch 3/15
229/229 [==============================] - ETA: 0s - loss: 1.1578 - auc: 0.8886
Epoch 3: val_auc improved from 0.90264 to 0.91582, saving model to ./best_weights_densenet169_401classes.hdf5
229/229 [==============================] - 451s 2s/step - loss: 1.1578 - auc: 0.8886 - val_loss: 0.9980 - val_auc: 0.9158
Epoch 4/15
229/229 [==============================] - ETA: 0s - loss: 1.0983 - auc: 0.8992
Epoch 4: val_auc improved from 0.91582 to 0.91792, saving model to ./best_weights_densenet169_401classes.hdf5
229/229 [======================

In [28]:
model_json = model1.to_json()

with open("model_behaviour_analysis_densenet169_401.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model1.save_weights("densent401.h5")

In [5]:
from tensorflow.keras.models import model_from_json

# Load the JSON model architecture from the file
with open(r'f:\emotion_facerecognition\final details of project_emotions\densnet\model_behaviour_analysis_densenet169_401 (2).json', 'r') as json_file:
    loaded_model_json = json_file.read()

# Load the model architecture
loaded_model = model_from_json(loaded_model_json)

# Load the model weights
loaded_model.load_weights(r"f:\emotion_facerecognition\final details of project_emotions\densnet\best_weights_densenet169_401classes.hdf5")

# Now you can use the loaded model for predictions

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
